In [ ]:
%load_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

num_of_classes = ds_info.features['label'].num_classes
image_shape = ds_info.features['image'].shape
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples

print("Number of classes:", num_of_classes)
print("Image shape:", image_shape)
print("Train size:", train_size)
print("Test size:", test_size)

def normalize_img(image, label):
  image = tf.cast(image, tf.float32) / 255.
  return image, label

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(train_size)
ds_train = ds_train.batch(64)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)


layers = [
    tf.keras.layers.Flatten(input_shape=image_shape),

    tf.keras.layers.Dense(21, kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.05),

    tf.keras.layers.Dense(8, kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(82, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(6, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),

    tf.keras.layers.Dense(num_of_classes),
    tf.keras.layers.Softmax()
]

model = tf.keras.models.Sequential(layers)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

model.summary()

EPOCHS = 40
history = model.fit(
    ds_train,
    epochs=EPOCHS,
    validation_data=ds_test
)

best_epoch_value = max(history.history['val_sparse_categorical_accuracy'])
best_epoch_index = history.history['val_sparse_categorical_accuracy'].index(best_epoch_value) + 1
print(f"Best performance at epoch {best_epoch_index} with validation sparse categorical accuracy of {best_epoch_value:.4f}")


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Number of classes: 10
Image shape: (28, 28, 1)
Train size: 60000
Test size: 10000


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_15 (Flatten)                 │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_75 (Dense)                     │ (None, 21)                  │          16,485 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_60               │ (None, 21)                  │              84 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_60 (Activation)           │ (None, 21)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_45 (Dropout)                 │ (None, 21)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_76 (Dense)                     │ (None, 8)                   │             176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_61               │ (None, 8)                   │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_61 (Activation)           │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_46 (Dropout)                 │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_77 (Dense)                     │ (None, 82)                  │             738 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_62               │ (None, 82)                  │             328 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_62 (Activation)           │ (None, 82)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_47 (Dropout)                 │ (None, 82)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_78 (Dense)                     │ (None, 6)                   │             498 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_63               │ (None, 6)                   │              24 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_63 (Activation)           │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_79 (Dense)                     │ (None, 10)                  │              70 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_15 (Softmax)                 │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,435 (72.01 KB)

 Trainable params: 18,201 (71.10 KB)

 Non-trainable params: 234 (936.00 B)

Epoch 1/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 1.5917 - sparse_categorical_accuracy: 0.4547 - val_loss: 0.6232 - val_sparse_categorical_accuracy: 0.8761
Epoch 2/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.8096 - sparse_categorical_accuracy: 0.7615 - val_loss: 0.4070 - val_sparse_categorical_accuracy: 0.9092
Epoch 3/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.6551 - sparse_categorical_accuracy: 0.8114 - val_loss: 0.3515 - val_sparse_categorical_accuracy: 0.9196
Epoch 4/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.5814 - sparse_categorical_accuracy: 0.8365 - val_loss: 0.3097 - val_sparse_categorical_accuracy: 0.9247
Epoch 5/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.5331 - sparse_categorical_accuracy: 0.8528 - val_loss: 0.3039 - val_sparse_categorical_accuracy: 0.9282
Epoch 6/40
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.5094 - sparse_categorical_accuracy: 0.8585 - val_loss: 0.2950 - val_sparse_categorical_accuracy: 0.9302
Epo